In [1]:
rdd = sc.textFile("taxi-data-sorted-small.csv.bz2", 1000)

In [2]:
rdd.take(5)

['07290D3599E7A0D62097A346EFCC1FB5,E7750A37CAB07D0DFF0AF7E3573AC141,2013-01-01 00:00:00,2013-01-01 00:02:00,120,0.44,-73.956528,40.716976,-73.962440,40.715008,CSH,3.50,0.50,0.50,0.00,0.00,4.50',
 '22D70BF00EEB0ADC83BA8177BB861991,3FF2709163DE7036FCAA4E5A3324E4BF,2013-01-01 00:02:00,2013-01-01 00:02:00,0,0.00,0.000000,0.000000,0.000000,0.000000,CSH,27.00,0.00,0.50,0.00,0.00,27.50',
 '0EC22AAF491A8BD91F279350C2B010FD,778C92B26AE78A9EBDF96B49C67E4007,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.71,-73.973145,40.752827,-73.965897,40.760445,CSH,4.00,0.50,0.50,0.00,0.00,5.00',
 '1390FB380189DF6BBFDA4DC847CAD14F,BE317B986700F63C43438482792C8654,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.48,-74.004173,40.720947,-74.003838,40.726189,CSH,4.00,0.50,0.50,0.00,0.00,5.00',
 '3B4129883A1D05BE89F2C929DE136281,7077F9FD5AD649AEACA4746B2537E3FA,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.61,-73.987373,40.724861,-73.983772,40.730995,CRD,4.00,0.50,0.50,0.00,0.00,5.00']

In [1]:
import bz2
with bz2.open("taxi-data-sorted-small.csv.bz2", 'rt') as f:
    content = f.readlines()

In [2]:
rdd = sc.parallelize(content[5000])

In [34]:
content[150000]

'26C5365B5F8A4495DA0A261F926C53DB,0A08EC79EF0E7F3C14DC4AD92D27F97F,2013-01-01 17:16:00,2013-01-01 17:18:00,120,0.56,-73.963966,40.808216,-73.966705,40.804272,CSH,4.00,0.00,0.50,0.00,0.00,4.50\n'

In [4]:
# Get cleaned rdd of text list
textLines = rdd.map(lambda x: x.rstrip("\n"))\
                .map(lambda x: x.split(","))\
                .filter(lambda x: len(x) == 17)

In [5]:
#1. Get top ten taxis that have had largest number of drivers
aggCount_taxis_drivers= textLines.map(lambda x: (x[0], x[1]))\
                                 .groupByKey()\
                                 .mapValues(len)

In [6]:
top_10_taxis_drivers = aggCount_taxis_drivers.top(10, key=lambda x: x[1])

In [7]:
top_10_taxis_drivers

[('F8A0B52B22BB58B3C45E66CEE135C29D', 4),
 ('9B5BE818E3C04896687FFD3C4BF5555F', 3),
 ('8669D9002FF4D63D433C5D9CCA710D7C', 3),
 ('6BA29E9A69B10F218C1509BEDD7410C2', 2),
 ('03B0493FEB9C714754477C4B816B7B73', 2),
 ('F6D3816AA5A0C24DE107AC490013796F', 2),
 ('D7DB65DF378CBC004BC9B41D223AAADF', 2),
 ('EDC498F8AB44E4697ECB239CF11E001F', 2),
 ('0964BAF015F74B4E628615EAFD0E2D57', 2),
 ('F2751CE89ED7E006D8E9C243FC31D01C', 2)]

In [8]:
# 2. Get top ten drivers who eanred more money per minute

In [9]:
# Calculate money earned per minutes for each driver
# First take index of 1'hack_license', 4'trip_time_in_secs', and 16'total_amount' from textLines.
# ReduceByKey with key of  hack_license' summing up 'trip_time_in_secs' in minutes 'total_amount'
# Map to avarage amount of earned money per minutes.

# earnByMinute = textLines.map(lambda x: (x[1], (float(x[4]), float(x[16]))))\
#                         .filter(lambda x: (x[0] != 0) and (x[1][0] != 0) and  )\
#                         .reduceByKey(lambda x, y: (x[0]/60 + y[0]/ 60, x[1] + y[1]))\
#                         .filter(lambda x: x != 0)\
#                         .map(lambda x: (x[0], x[1][1] / x[1][0]))

In [10]:
# earnByMinute.take(3)

In [11]:
# Calculate total duration driven by each driver
# Filter 0 out
totalMinute = textLines.map(lambda x: (x[1], float(x[4])))\
                        .filter(lambda x: 0 not in x)\
                        .reduceByKey(lambda x, y: x/60 + y/60)

In [12]:
# Calculate total money earned by each driver
# Filter 0 out
totalMoney = textLines.map(lambda x: (x[1], float(x[16])))\
                      .filter(lambda x: 0 not in x)\
                      .reduceByKey(lambda x, y: x + y)

In [13]:
# Calculate earning per minute joining total
# Filter 0 our
earningPerMinute = totalMinute.join(totalMoney)\
                              .filter(lambda x: 0 not in x[1])\
                              .map(lambda x: (x[0], x[1][1] / x[1][0]))

In [14]:
earningPerMinute.top(10, key=lambda x: x[1])

[('00B7691D86D96AEBD21DD9E138F90840', 581.3938132986634),
 ('4D85F9862CD0E1675B0946F1A4535B42', 8.20625),
 ('FD717A920A5125D904090815FA489598', 5.585106382978723),
 ('8322F2222F5D9F5E23FB0A9D866EAF7E', 4.233333333333333),
 ('DD9ABBF45EC86DFD0E96AAAAB55AF385', 2.0833333333333335),
 ('0D5AC420758EA96775D233FC8529983A', 2.033333333333333),
 ('2E0CA2421982DDF5DC7574CFA58DBC84', 1.9944444444444445),
 ('108B18241CE401E06AAC4CF376735BF2', 1.957142857142857),
 ('69D19CD8ADCAA46F9B8AF942D3F67C31', 1.9285714285714286),
 ('CF8604E72D83840FBA1978C2D2FC9CDB', 1.8857142857142857)]

In [15]:
# 3. Find the hour of the day when has the best profite ration

In [19]:
from datetime import datetime
import pandas as pd

In [20]:
# Function to 
def getRange(start, end, t):
    time_range = pd.date_range(start, end, freq = "H" )
    if t == "date":
        return time_range.date
    if t == "hour":
        return time_range.hour

In [24]:
# flat date, hour, surgent and distance by hour
# to ((datetime.date, hour), (trip_distance, surcharge))
flatHour = textLines.map(lambda x: (x[2], x[3], x[5], x[12]))\
        .flatMap(lambda x: [( (d, int(h)) ,(float(x[2]), float(x[3]))) \
                            for d, h in zip(getRange(x[0], x[1], "date"),\
                                            getRange(x[0], x[1], "hour"))])

In [54]:
def getTime(d, t):
    if t == "date":
        return datetime.strptime(d, "%Y-%m-%d %H:%M:%S").date()
    if t == "hour":
        return datetime.strptime(d, "%Y-%m-%d %H:%M:%S").hour

In [57]:
hour = textLines.map(lambda x: ((getTime(x[2], "date"), getTime(x[2], "hour")), 
                                (float(x[5]), float(x[12]))))\
                .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
                .map(lambda x: (x[0], x[1][1] / x[1][0]))\
                .map(lambda x: (x[0][0], (x[1], x[0][1])))\
                .reduceByKey(max)

In [58]:
hour.take(5)

[(datetime.date(2013, 1, 1), (0.31521407892731973, 0))]

In [26]:
# culculate ratio and map to key datetime.date and value (ratio, hour)
# reduce by the key datetime.data and take max of tuple based on ratio
bestHourForDay = flatHour.reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
                          .map(lambda x: (x[0], x[1][1] / x[1][0]))\
                          .map(lambda x: (x[0][0], (x[1], x[0][1])))\
                          .reduceByKey(max)

In [27]:
bestHourForDay.take(5)

[((datetime.date(2013, 1, 1), 0), (1575.1200000000003, 496.5))]

In [ ]:
test -

In [41]:
getRange("2013-01-01 00:15:00", "2013-01-01 00:15:00", "hour")

Int64Index([0], dtype='int64')

In [45]:
datetime.strptime("2013-01-01 00:15:00", "%Y-%m-%d %H:%M:%S").()

TypeError: 'int' object is not callable